In [417]:
import os
import ast
from tqdm import tqdm
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [387]:
path = '/Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/jieun'
rpath = os.path.join(path, 'tmp.csv')
tmp = pd.read_csv(rpath)
tmp

,text,result
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,"[[{'label': '기쁨(행복한)', 'score': 0.628699421882..."
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,"[[{'label': '기쁨(행복한)', 'score': 0.741971969604..."
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,"[[{'label': '기쁨(행복한)', 'score': 0.048555459827..."
3,아파트는 사진에서 보이는 것보다 더 넓습니다,"[[{'label': '기쁨(행복한)', 'score': 0.012977219186..."
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,"[[{'label': '기쁨(행복한)', 'score': 0.075244806706..."
...,...,...
2995,시간 지하철이 통과합니다,"[[{'label': '기쁨(행복한)', 'score': 0.019679322838..."
2996,윌리엄은 매우 친절했습니다,"[[{'label': '기쁨(행복한)', 'score': 0.069694191217..."
2997,온라인에 있는 침실 사진이 오래되었습니다,"[[{'label': '기쁨(행복한)', 'score': 0.013530771248..."
2998,훌륭한 위치 특히 햇살이 잘 드는 거실 공간과 발코니로 연결된 미닫이문이 마음에 들...,"[[{'label': '기쁨(행복한)', 'score': 0.647638738155..."


In [388]:
#* 데이터프레임 전처리1
for idx in range(len(tmp)):
    tmp['result'][idx] = ast.literal_eval(tmp['result'][idx])

In [389]:
a = tmp['result'][3][0]
maxScore = 0
for idx, value in enumerate(tmp['result'][3][0]):
    if value['score'] > maxScore:
        maxScore = value['score']
        maxlabel = value['label']
print(maxScore, maxlabel)

0.7937557101249695 일상적인


In [390]:
maxScores = [] 
maxEmotions = []  

for idx in range(len(tmp)):
    maxScore = 0
    maxEmotion = ''
    for emotion_data in tmp['result'][idx][0]:
        score = emotion_data['score']
        if score > maxScore:
            maxScore = score
            maxEmotion = emotion_data['label']
    maxScores.append(maxScore)
    maxEmotions.append(maxEmotion)


tmp['MaxScore'] = maxScores
tmp['MaxEmotion'] = maxEmotions
tmp = tmp.drop(columns=['result'])

def emotionProcessing(row, idx):
    row['MaxEmotion'] = row['MaxEmotion'].split('(')[0]
    return row

for idx in range(11):
    tmp = tmp.apply(lambda row: emotionProcessing(row, idx), axis=1)

final = pd.DataFrame(tmp)
final.head()

,text,MaxScore,MaxEmotion
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,0.628699,기쁨
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,0.741972,기쁨
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,0.511306,일상적인
3,아파트는 사진에서 보이는 것보다 더 넓습니다,0.793756,일상적인
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,0.696043,일상적인


In [394]:
# fpath = os.path.join(path, 'maxEmotion.csv')
# final.to_csv(fpath, index=False)

In [391]:
# for idx in range(len(tmp)):
#     for cnt in range(11):
#         tmp.loc[idx, f'Emotion{cnt + 1}'] = tmp['result'][idx][0][cnt]['label']
#         tmp.loc[idx, f'EmotionProb{cnt+ 1}'] = tmp['result'][idx][0][cnt]['score']
# tmp.drop(columns=['result'], inplace=True)

# #* 데이터프레임 전처리2
# #TODO : 기쁨(행복한) -> 기쁨
# def emotionProcessing(row, idx):
#     row[f'Emotion{idx + 1}'] = row[f'Emotion{idx + 1}'].split('(')[0]
#     return row

# for idx in range(11):
#     tmp = tmp.apply(lambda row: emotionProcessing(row, idx), axis=1)

# final = pd.DataFrame(tmp)

In [395]:
final

,text,MaxScore,MaxEmotion
0,더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다,0.628699,기쁨
1,게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼...,0.741972,기쁨
2,처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다,0.511306,일상적인
3,아파트는 사진에서 보이는 것보다 더 넓습니다,0.793756,일상적인
4,커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다,0.696043,일상적인
...,...,...,...
2995,시간 지하철이 통과합니다,0.854543,일상적인
2996,윌리엄은 매우 친절했습니다,0.392459,일상적인
2997,온라인에 있는 침실 사진이 오래되었습니다,0.739786,일상적인
2998,훌륭한 위치 특히 햇살이 잘 드는 거실 공간과 발코니로 연결된 미닫이문이 마음에 들...,0.647639,기쁨


In [429]:
wordList = tmp['MaxEmotion'].tolist()
wordCount = Counter(wordList)
wordCountdf = pd.DataFrame.from_dict(wordCount, orient='index').reset_index()
wordCountdf.columns = ['emotion', 'count']
wordCountdf = wordCountdf.sort_values(by='count', ascending=False).reset_index(drop=True)
wordCountdf

,emotion,count
0,기쁨,1227
1,일상적인,851
2,고마운,445
3,생각이 많은,138
4,설레는,134
5,슬픔,96
6,즐거운,63
7,짜증남,18
8,사랑하는,16
9,걱정스러운,11


In [442]:
import plotly.express as px

fig = px.sunburst(
    wordCountdf,
    path = ['emotion', 'count'],
    labels = 'emotion',
    values = 'count',
    width= 600 , height= 600
)
fig.show()